#Single-fidelity GP surrogate model

In the following notebook construct, train, and validate a single-fidelity Gaussian Process (GP) surrogate model of lung function.

We begin by importing the necessary modules.

In [1]:
from IPython.display import clear_output

In [2]:
# Install Emukit (useful for the construction of our surrogate model)
!pip install emukit
clear_output()
print('Emukit installed')

Emukit installed


In [3]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
%matplotlib inline

# Emukit imports
import GPy
import emukit.multi_fidelity
import emukit.test_functions
import emukit.test_functions.multi_fidelity
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel

np.random.seed(20)

## Import high-fidelity dataset

Next, we import and load the created dataset from high fidelity simulations. Each sample/observations of the dataset consists in an input and output array of the form:

\begin{array}{cc}
\text{Input} & \text{Output}\\
[ c, \beta, c_1, c_3, k, K_s ] & [ \mathrm{C_{rs}}, \mathrm{R} ]
\end{array}
\
,where the input contains our six model parameters ($c$, $\beta$, $c_1$ and $c_3$ from the tissue constitutive model + permeability $k$ + chest wall stiffness $K_s$) and the output our two variables that represent the lung response (respiratory system compliance $\mathrm{C_{rs}}$ and airways resistance $\mathrm{R}$).

\
- For the surrogate model training, we have 20 samples.

- For the surrogate model testing, we have 100 samples.

In [4]:
# Getting files from repository
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/training_input_data_20_hf.npy
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/training_output_data_20_hf.npy
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/testing_input_data_100_hf.npy
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/testing_output_data_100_hf.npy

clear_output()

In [5]:
# High Fidelity training datasets
X_hf = np.load('training_input_data_20_hf.npy')
Y_hf = np.load('training_output_data_20_hf.npy')

# High Fidelity testing datasets
X_hf_100 = np.load('testing_input_data_100_hf.npy')
Y_hf_100 = np.load('testing_output_data_100_hf.npy')

print("Checking size of HF and LF datasets:")
print("")
print("Training X HF size :", X_hf.shape)
print("Training Y HF size :", Y_hf.shape)
print("")
print("Testing X HF size :", X_hf_100.shape)
print("Testing Y HF size :", Y_hf_100.shape)

Checking size of HF and LF datasets:

Training X HF size : (20, 6)
Training Y HF size : (20, 2)

Testing X HF size : (100, 6)
Testing Y HF size : (100, 2)


## Data preprocessing for single-fidelity Gaussian Process (GP) model

We divide our training dataset into an internal training and validation sets. As there are only a few observations, we want to use the majority of them for training our surrogate model. Thus, we specify a training size of 95%, leaving only one observation for the internal model validation.The testing dataset remains untouched for the entire analysis and is only used to evaluate the performance after we trained our surrogate model.

In [6]:
from sklearn.model_selection import train_test_split
# Next, the High Fidelity training datasets is split into an internal Training and Validation set
X_hf_train, X_hf_test = train_test_split(X_hf, train_size=0.95, shuffle=True, random_state=1) # this split the input data
Y_hf_train, Y_hf_test = train_test_split(Y_hf, train_size=0.95, shuffle=True, random_state=1) # this split the output data

X_hf_test_100 = X_hf_100
Y_hf_test_100 = Y_hf_100

Given that the six model parameters have considerable differences in their scale, it is necessary to normalize the input dataset:

In [7]:
from sklearn.preprocessing import StandardScaler

## Normalize input data
scaler = StandardScaler()

X_hf_train = scaler.fit_transform(X_hf_train)
X_hf_test = scaler.transform(X_hf_test)
X_hf_test_100 = scaler.transform(X_hf_test_100)

## ML surrogate model construction

After the data is prepared, we construct and train the single-fidelity GP model.

Regarding this, we want to learn a latent function $f$ from our dataset, such that $\boldsymbol{y}=f(\boldsymbol{X})+\epsilon$.

First, we establish that our prior knowledge about $f$ can be described by a GP with mean $m(\boldsymbol{x})$ and covariance $k(\boldsymbol{x}, \boldsymbol{x}^{\prime} ; \boldsymbol{\theta})$,

\begin{equation}
    f(\boldsymbol{x}) \sim \mathcal{G} \mathcal{P}\left(m(\boldsymbol{x}), k(\boldsymbol{x}, \boldsymbol{x}^{\prime} ; \boldsymbol{\theta})\right),
\end{equation}

where we assume a zero-mean function $m(\boldsymbol{x}) = 0$. In our prior, $\boldsymbol{\theta}$ denotes the hyperparameters of the covariance function or kernel, which encodes our assumption about the Gaussian process. Here, we choose a quadratic exponential kernel (RBF), expressed in the form

\begin{equation}
    k(\boldsymbol{x},\boldsymbol{x}';\boldsymbol{\theta})=v\exp\left\{-\sum\limits_{i=1}^d\dfrac{\left(x_i-x_i'\right)^2}{2\ell_i^2}\right\},       
\end{equation}

where $v$ y $\{\ell_i\}_{i=1,\ldots,d}$ are the model hyperparameters: $v$ models the variance of the process, and $\ell_i$ determines the lengthscale of each input dimension.

After training the model, we compute the prediction of the posterior distribution,

\begin{equation}
    p(\boldsymbol{y}^{*}|\boldsymbol{x}^{*},\mathcal{D})\sim\mathcal{N}(\mu(\boldsymbol{x}^{*}),\varSigma(\boldsymbol{x}^{*}))
\end{equation}

\
From here, it is important to note that we build two separate models: one for the respiratory-system compliance prediction, and the other for the airways resistance.

In [8]:
input_dim = X_hf_train.shape[1]
Y_hf_train_r = Y_hf_train[:,0][:,np.newaxis]
Y_hf_train_c = Y_hf_train[:,1][:,np.newaxis]

## Construct a single-fidelity model
kernel_r = GPy.kern.RBF(input_dim, ARD = True)
kernel_c = GPy.kern.RBF(input_dim, ARD = True)

### RESISTANCE model ###
m_r = GPy.models.GPRegression(X_hf_train, Y_hf_train_r, kernel_r)
### COMPLIANCE model ###
m_c = GPy.models.GPRegression(X_hf_train, Y_hf_train_c, kernel_c)

## Fit (optimize) the models
#m.optimize(optimizer='scg', max_iters=1000)
m_r.optimize()
print('Resistance model optimized')
m_c.optimize()
print('Compliance model optimized')

 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in true_divide
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Resistance model optimized
Compliance model optimized


After trained, we evaluate the model performance and save the model predictions on the HF testing set

In [9]:
# Predictions
Y_pred_mean_R, Y_pred_var_R = m_r.predict(X_hf_test_100)
Y_pred_mean_C, Y_pred_var_C = m_c.predict(X_hf_test_100)

# Performance
print(r2_score(Y_hf_test_100[:,0], Y_pred_mean_R[:,0]))
print(r2_score(Y_hf_test_100[:,1], Y_pred_mean_C[:,0]))

# Export values
np.save('sfGP_var_C.npy', Y_pred_var_C)
np.save('sfGP_var_R.npy', Y_pred_var_R)
np.save('sfGP_C.npy', Y_pred_mean_C)
np.save('sfGP_R.npy', Y_pred_mean_R)

0.9986261287224183
0.709755788192304


## Cross-Validation (in testing set)

Additionally, we perform a cross-validation using different training sizes.

**Note**: *Take consideration that the following code block may take several hours to be executed*.

In [11]:
split_list = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
random_list = np.arange(1, 100+1,10)
RMSE_list_R = []
RMSE_list_C = []
std_R = []
std_C = []

for split in split_list:
  print("Train split: ", split)
  RMSE_inner_list_R = []
  RMSE_inner_list_C = []
  for random_number in random_list:
    # Split
    X_hf_train, X_hf_test = train_test_split(X_hf, train_size=split, shuffle=True, random_state=random_number) # this split the input data
    Y_hf_train, Y_hf_test = train_test_split(Y_hf, train_size=split, shuffle=True, random_state=random_number) # this split the output data

    X_hf_test_100 = X_hf_100
    Y_hf_test_100 = Y_hf_100

    # Normalize
    scaler = StandardScaler()

    X_hf_train = scaler.fit_transform(X_hf_train)
    X_hf_test = scaler.transform(X_hf_test)
    X_hf_test_100 = scaler.transform(X_hf_test_100)

    # Train
    input_dim = X_hf_train.shape[1]
    Y_hf_train_r = Y_hf_train[:,0][:,np.newaxis]
    Y_hf_train_c = Y_hf_train[:,1][:,np.newaxis]

    ## Construct a single-fidelity model
    kernel_r = GPy.kern.RBF(input_dim, ARD = True)
    kernel_c = GPy.kern.RBF(input_dim, ARD = True)
    m_r = GPy.models.GPRegression(X_hf_train, Y_hf_train_r, kernel_r)
    m_c = GPy.models.GPRegression(X_hf_train, Y_hf_train_c, kernel_c)
    #print(m)
    #m.optimize(optimizer='scg', max_iters=1000)
    m_r.optimize()
    m_c.optimize()
    #print(m)

    # Evaluate
    Y_pred_mean_r, Y_pred_var_r = m_r.predict(X_hf_test_100)
    Y_pred_mean_c, Y_pred_var_c = m_c.predict(X_hf_test_100)
    # H-F predictions
    print("H-F predictions")
    mse_R = mean_squared_error(Y_hf_test_100[:,0], Y_pred_mean_r[:,0])
    print('\nMSE R: {}'.format(mse_R))
    mse_C = mean_squared_error(Y_hf_test_100[:,1], Y_pred_mean_c[:,0])
    print('\nMSE C: {}'.format(mse_C))
    RMSE_inner_list_R.append(np.sqrt(mse_R))
    RMSE_inner_list_C.append(np.sqrt(mse_C))
  RMSE_list_R.append(np.average(RMSE_inner_list_R))
  RMSE_list_C.append(np.average(RMSE_inner_list_C))
  print(RMSE_inner_list_R)
  print(RMSE_inner_list_C)
  std_R.append(np.std(RMSE_inner_list_R))
  std_C.append(np.std(RMSE_inner_list_C))

#clear_output()

for i in range(len(split_list)):
  print("Train split: ", split_list[i], "||    MSE R: ", RMSE_list_R[i], "||    MSE C: ", RMSE_list_C[i], "||    std R: ", std_R[i], "||    std C: ", std_C[i])

Train split:  0.5
H-F predictions

MSE R: 0.009253323258857735

MSE C: 222.47780963953343
H-F predictions

MSE R: 0.02373573723338636

MSE C: 933.3289635414155
H-F predictions

MSE R: 0.009754944016520647

MSE C: 230.0715564203132


 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide


H-F predictions

MSE R: 0.007730535127435738

MSE C: 293.7422989460658


 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


H-F predictions

MSE R: 0.030737964557902234

MSE C: 729.9706270845331
H-F predictions

MSE R: 0.010423192102922908

MSE C: 236.1090785562649


...

H-F predictions

MSE R: 0.0012318894525230213

MSE C: 209.33595240184889


 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide


H-F predictions

MSE R: 0.0025549518609189765

MSE C: 109.74464966447336
H-F predictions

MSE R: 0.0019415988411337277

MSE C: 108.39301790207745
H-F predictions

MSE R: 0.001891500190984246

MSE C: 98.96106778768744
H-F predictions

MSE R: 0.004374255437911688

MSE C: 96.30182690683601
H-F predictions

MSE R: 0.0019524079436978202

MSE C: 96.06387443292924


 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:overflow encountered in square
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add
 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


H-F predictions

MSE R: 0.0012319120943160432

MSE C: 209.33595242358535
H-F predictions

MSE R: 0.005013024908124266

MSE C: 195.91114719026376
[0.04752148632854557, 0.04752226407497581, 0.03509828275746581, 0.05054653164084531, 0.04406357726210762, 0.0434913806516216, 0.06613815417678126, 0.044186060513444966, 0.035098605304428314, 0.0708027182255333]
[14.129986854655469, 14.129956210071253, 14.46844678608761, 10.47590805918386, 10.411196756477011, 9.947917761405522, 9.813349423455582, 9.801218007621769, 14.468446786838777, 13.996826325644816]
Train split:  0.5 ||    MSE R:  0.11121055743323845 ||    MSE C:  21.0225381888227 ||    std R:  0.03166965807239767 ||    std C:  5.510692321106828
Train split:  0.55 ||    MSE R:  0.08614019191824598 ||    MSE C:  20.99999035912665 ||    std R:  0.025052434121357717 ||    std C:  6.725316689123867
Train split:  0.6 ||    MSE R:  0.09029536730534118 ||    MSE C:  19.201811056437936 ||    std R:  0.02608419424679336 ||    std C:  5.987548401464

In [12]:
# Export values
np.save('RMSE_R_sfGP.npy', RMSE_list_R)
np.save('RMSE_C_sfGP.npy', RMSE_list_C)
np.save('std_R_sfGP', std_R)
np.save('std_C_sfGP', std_C)